In [1]:
import tkinter as Tk
from PIL import Image, ImageTk, ImageDraw, ImageFont
import os
import numpy as np
import collections
import csv
from random import randint
import pandas as pd

In [72]:
clipNum = 'CLIP_011'

In [73]:
cwd = os.getcwd()
datadir = os.path.join(cwd, 'data', clipNum, 'FRAMES')

n = len(os.listdir(datadir))
currImg = 1

positions = {}

radius = 50

In [74]:
print n

710


In [5]:
def onButtonPress(event):
    canvas = event.widget

    x = canvas.canvasx(event.x)
    y = canvas.canvasy(event.y)
    
    if len(canvas.find_all()) == 2:
        oldPos = canvas.coords(2)
        canvas.move(2, x-radius-oldPos[0], y-radius-oldPos[1])
    else:
        circle = canvas.create_oval(x-radius, y-radius, 
                                    x+radius, y+radius,
                                    outline="#f11", width=2)
    
def onEsc(event, root, imageNum):
    global positions
    canvas = event.widget
    if len(canvas.find_all()) == 2:
        positions[imageNum] = canvas.coords(2)
        #print positions[imageNum]
        
    root.destroy()
    
def size(event, mode='up'):
    canvas = event.widget
    
    if len(canvas.find_all()) == 2:
        oldPos = canvas.coords(2)
        
        if mode == 'up':
            canvas.coords(2, oldPos[0], oldPos[1] - 10, oldPos[2], oldPos[3] + 10)
        
        if mode == 'down':
            canvas.coords(2, oldPos[0], oldPos[1] + 10, oldPos[2], oldPos[3] - 10)
            
        if mode == 'right':
            canvas.coords(2, oldPos[0] - 10, oldPos[1], oldPos[2] + 10, oldPos[3])
            
        if mode == 'left':
            canvas.coords(2, oldPos[0] + 10, oldPos[1], oldPos[2] - 10, oldPos[3])
    
    
def showFirst(event):
    path = os.path.join(datadir, 'frame_%04d.png' % (start))
    root = Tk.Tk()

    canvas = Tk.Canvas(root, cursor="cross")
    canvas.pack(fill=Tk.BOTH, expand=1) # Stretch canvas to root window size.
    canvas.focus_set()
    canvas.bind("<h>", lambda event: destroyFirst(event, root))

    background_image = ImageTk.PhotoImage(Image.open(path).resize((1200,700)), master=root)
    image = canvas.create_image(0, 0, anchor=Tk.NW, image=background_image)

    guess = positions[start]
    circle = canvas.create_oval(guess[0],guess[1],guess[2],guess[3],
                               outline="#f11", width=2)
    root.wm_geometry("1200x700")
    root.title('First frame')
    root.mainloop()
                

def destroyFirst(event, root):
    root.destroy()
    
def quit(root):
    global stop 
    stop = True
    root.destroy()

def showImage(imageNum, oldImageNum):
    root = Tk.Tk()

    canvas = Tk.Canvas(root, cursor="cross")
    canvas.pack(fill=Tk.BOTH, expand=1) # Stretch canvas to root window size.
    canvas.focus_set()
    canvas.bind("<Button-1>", onButtonPress)
    canvas.bind("<space>", lambda event: onEsc(event, root, imageNum))
    canvas.bind("<Up>", lambda event: size(event, mode='up'))
    canvas.bind("<Down>", lambda event: size(event, mode='down'))
    canvas.bind("<Left>", lambda event: size(event, mode='left'))
    canvas.bind("<Right>", lambda event: size(event, mode='right'))
    canvas.bind("<h>", lambda event: showFirst(event))
    canvas.bind("<q>", lambda event: quit(root))
    
    if started and imageNum == start:
        path = os.path.join('data/'+ clipNum +'/CLUSTERED_FRAMES', 'frame_%04d.png' % (imageNum))
    else:
        path = os.path.join(datadir, 'frame_%04d.png' % (imageNum))

    
    background_image = ImageTk.PhotoImage(Image.open(path).resize((1200,700)))
    image = canvas.create_image(0, 0, anchor=Tk.NW, image=background_image)
    
    if oldImageNum in positions:
        guess = positions[oldImageNum]
        circle = canvas.create_oval(guess[0],guess[1],guess[2],guess[3],
                                    outline="#f11", width=2)
    root.wm_geometry("1200x700")
    root.title('Frame %d' % imageNum)
    root.mainloop()

In [6]:
def getClusters(start, end, step=800):
    for frame in range(start, end+1, step):
        showImage(frame, frame-step)
        if stop: break
    
    showImage(end, frame)

In [7]:
def backfillPositions(positions, first, last):
    #after tracking, fill in the rest of the positions
    fullPositions = positions.copy()
    od = collections.OrderedDict(sorted(positions.items()))
    for frame, pos in od.iteritems(): 
        if frame == first: 
            lastFrame = first
            continue
        if lastFrame + 1 == frame:
            lastFrame = frame
            continue
        else: 
            #print ("doing interpolating")
            #some gap between tmpFrame (last frame seen) and curFrame
            stepSize = np.subtract(pos, positions[lastFrame])/(frame - lastFrame)
            for i in range(lastFrame + 1, frame):
                fullPositions[i] = list(np.add(stepSize, fullPositions[i-1]))
            lastFrame = frame
            
    if first > 1: 
        for i in range(1, first):
            fullPositions[i] = [1166.0, 656.0, 1206.0, 696.0]
    
    if end < 710: 
        for i in range(last + 1, 711):
            fullPositions[i] = [1166.0, 656.0, 1206.0, 696.0]
            
    fullSorted = collections.OrderedDict(sorted(fullPositions.items()))
    return fullSorted

In [8]:
def saveData(datafile, positions):
    with open (datafile, 'a') as csvfile: 
        writer = csv.writer(csvfile, delimiter=',')
        #Comment out after editing
        writer.writerow([])
        data = []
        for frame, pos in positions.iteritems():
            data.append(pos)
        writer.writerow(data)

In [26]:
def saveFirstFrame(sourceDir, destDir, positionsFile, start):
    #take all positions and overlay on images, save as video
    
    df = pd.read_csv(positionsFile, header=None)
    df.dropna(inplace=True)
    colors = [(195, 195, 195) for x in df.index]
    font = ImageFont.truetype("VCR_OSD_MONO_1.001.ttf", 16)
    for col in df.columns[start:start+1]: 
        path = os.path.join(sourceDir, 'frame_%04d.png' % (col))
        newPath = os.path.join(destDir, 'frame_%04d.png' % (col))

        img = Image.open(path, 'r')
        img = img.resize((1200,700))
        img_w, img_h = img.size
        background = Image.new('RGBA', (img_w, img_h), (255, 255, 255, 255))
        background.paste(img, (0,0))
        draw = ImageDraw.Draw(background)
        draw.text((20,20), str(col), (255,0,0), font=font)
        for ID, person in enumerate(df[col]):
            coords = [float(x) for x in person[1:-1].split(',')]
            draw.ellipse(coords, outline=colors[ID], width=2)
            draw.text((coords[0], coords[1]), str(ID), colors[ID], font=font)
        background.save(newPath)

---
## DO NOT EDIT ABOVE THIS LINE

In [75]:
started = False

In [77]:
start = 1
end = n

stepSize = 50

In [78]:
positions = {}

stop = False
if started: 
    saveFirstFrame('data/' + clipNum + '/FRAMES', 
                   'data/'+ clipNum +'/CLUSTERED_FRAMES', 
                   'data/'+ clipNum +'/DATA/clusters_positions.csv',
                    start)

getClusters(start, end, step=stepSize)

fullTest = backfillPositions(positions, start, end)

In [79]:
saveData('data/' + clipNum + '/DATA/clusters_positions.csv', fullTest)